## Observations
1. Our gradient estimates at known sample locations are large when we resample at the sample location in our trajectory.

In [1]:
using LinearAlgebra
using Plots
using Random

In [2]:
include("../rollout.jl")
include("../testfns.jl")
include("../utils.jl")

stochastic_gradient_ascent_adam (generic function with 1 method)

In [3]:
# testfn = TestLinearCosine1D(1, 25, lb=0., ub=2)
# testfn = TestQuadratic1D(1., 0., 0.; lb=-1., ub=1.)
# testfn = TestConstant(0., lbs=[0.], ubs=[1.])
# testfn = TestGramacyLee()
testfn = TestSchwefel(4)
# tplot(testfn)

TestFunction(4, [-500.0 500.0; -500.0 500.0; -500.0 500.0; -500.0 500.0], ([420.9687, 420.9687, 420.9687, 420.9687],), var"#f#806"{Int64}(4), var"#∇f#807"{Int64}(4))

In [13]:
Random.seed!(1906)

n, h, σn2 = 1, 1, 1e-6
MC_SAMPLES = 25
# Setup low discrepancy random number stream
lds_rns = gen_low_discrepancy_sequence(MC_SAMPLES, testfn.dim, h+1);
rns = randn(MC_SAMPLES, testfn.dim+1, h+1);
xmin, xmax, d = (testfn.bounds[1], testfn.bounds[2], size(testfn.bounds, 1))
θ = [1.]
ψ = kernel_matern52(θ)
# ψ = kernel_scale(kernel_matern52, [5., θ...])

lbs, ubs = testfn.bounds[:,1], testfn.bounds[:,2]
ϵ, num_starts = 1e-6, 4
s = SobolSeq(lbs, ubs)

xstarts = reduce(hcat, next!(s) for i = 1:num_starts)
xstarts = hcat(xstarts, lbs .+ ϵ)
xstarts = hcat(xstarts, ubs .- ϵ);

### Dense Experiments in 1D for Rollout Acquisition Functions
We'll do a comparative analysis, visually, of the structure of the rollout acquisition function in 1D for horizons 0 and 1.

In [14]:
X = [-391.15754060162743 0.0 0.0; 218.9031628453905 0.0 0.0; 82.66621969601522 0.0 0.0; 431.0582938270935 0.0 0.0]
y = testfn(X)
sur = fit_surrogate(ψ, X, y; σn2=σn2)
# sur = optimize_hypers_optim(sur, kernel_matern52)

# print("Rollout Evaluations: ")
# # Dense Evaluation of Rollout Acquisition Function
# for (i, x) in enumerate(domain)
#     print("|")
#     x0 = [x]
#     sx = sur(x0)

#     tp = TrajectoryParameters(
#         x0=x0, h=h, mc_iters=MC_SAMPLES,
#         rnstream_sequence=lds_rns, lbs=lbs, ubs=ubs
#     )
#     # Monte-carlo integrate trajectory for x0
#     μx, ∇μx, μ_stderr, ∇μ_stderr = simulate_trajectory(sur, tp, xstarts, variance_reduction=variance_reduction)
# end

RBFsurrogate(RBFfun([1.0], var"#ψ#451"{var"#k#461", Vector{Float64}}(var"#k#461"(), [1.0]), var"#Dρ_ψ#452"{var"#ψ#451"{var"#k#461", Vector{Float64}}}(var"#ψ#451"{var"#k#461", Vector{Float64}}(var"#k#461"(), [1.0])), var"#Dρρ_ψ#453"{var"#Dρ_ψ#452"{var"#ψ#451"{var"#k#461", Vector{Float64}}}}(var"#Dρ_ψ#452"{var"#ψ#451"{var"#k#461", Vector{Float64}}}(var"#ψ#451"{var"#k#461", Vector{Float64}}(var"#k#461"(), [1.0]))), var"#∇θ_ψ#454"{var"#k#461", Vector{Float64}}(var"#k#461"(), [1.0])), [-391.15754060162743 0.0 0.0; 218.9031628453905 0.0 0.0; 82.66621969601522 0.0 0.0; 431.0582938270935 0.0 0.0], [1.000001 0.0 0.0; 0.0 1.000001 1.0; 0.0 1.0 1.000001], [1.000000499999875 0.0 0.0; 0.0 1.000000499999875 0.0; 0.0 0.999999500000375 0.0014142132088085626], [-195.4639666064238, 97.73198330321179, 97.73198330321179], [-195.46377114265263, 48.86596721281304, 48.86596722443153], 1.0e-6, 1578.199616696788)

In [37]:
"""
TODO: We need to specify the maximum number of iterations and terminate if we exhaust our budget
TODO: EI for Rosenbrock looks like zeros everywhere, depending on how we sample. I suspect this
is why our algorithm halts here.
"""
function ei_solver(s::RBFsurrogate, lbs, ubs; initial_guesses, max_iterations=100)
    fbest = minimum(get_observations(s))

    function ei(x)
        sx = s(x)
        if sx.σ < 1e-6 return 0 end
        return -sx.EI
    end

    function ei_grad!(g, x)
        EIx = -s(x).∇EI
        for i in eachindex(EIx)
            g[i] = EIx[i]
        end
    end

    function ei_hessian!(h, x)
        HEIx = -s(x).HEI
        for row in 1:size(HEIx, 1)
            for col in 1:size(HEIx, 2)
                h[row, col] = HEIx[row, col]
            end
        end
    end

    final_minimizer = (initial_guesses[:, 1], Inf)
    
    for j in 1:size(initial_guesses, 2)
        initial_guess = initial_guesses[:, j]
        df = TwiceDifferentiable(ei, ei_grad!, ei_hessian!, initial_guess)
        dfc = TwiceDifferentiableConstraints(lbs, ubs)
        result = optimize(df, dfc, initial_guess, IPNewton(), Optim.Options(iterations=max_iterations))
        cur_minimizer, cur_minimum = Optim.minimizer(result), Optim.minimum(result)

        if cur_minimum < final_minimizer[2]
            final_minimizer = (cur_minimizer, cur_minimum)
        end
    end
    
    return final_minimizer
end

function rollout_solver(;
    sur::RBFsurrogate,
    tp::TrajectoryParameters,
    xstarts::Matrix{Float64},
    batch::Matrix{Float64},
    max_iterations::Int = 25,
    varred::Bool = true,
    )
    batch_results = Array{Any, 1}(undef, size(batch, 2))

    for i in 1:size(batch, 2)
        # Update start of trajectory for each point in the batch
        tp.x0 = batch[:, i]

        # Perform stochastic gradient ascent on the point in the batch
        batch_results[i] = stochastic_gradient_ascent_adam(
            sur=sur,
            tp=tp,
            max_sgd_iters=max_iterations,
            varred=varred,
            xstarts=xstarts,
        )
    end

    # Find the point in the batch that maximizes the rollout acquisition function
    println("Number of Results: $(length(batch_results))")
    best_tuple = first(batch_results)
    for result in batch_results[2:end]
        if result.final_obj > best_tuple.final_obj
            best_tuple = result
        end
    end

    return best_tuple.finish, best_tuple.final_obj
end

rollout_solver (generic function with 1 method)

In [42]:
# Allocate all initial samples
initial_samples = randsample(1, testfn.dim, lbs, ubs)

# Initialize the trajectory parameters
tp = TrajectoryParameters(
    initial_samples[:, 1], # Will be overriden later
    h,
    MC_SAMPLES,
    lds_rns,
    lbs,
    ubs,
)
batch = shuffle(generate_batch(16, lbs=tp.lbs, ubs=tp.ubs))

4×18 Matrix{Float64}:
  499.99   375.0   312.5  -250.0  …   -31.25   250.0    437.5   -312.5
  312.5   -187.5  -125.0     0.0      -31.25  -187.5    250.0   -250.0
 -499.99   125.0     0.0   125.0       62.5    499.99  -187.5    -62.5
  -62.5     62.5   375.0  -125.0     -375.0   -312.5    156.25   499.99

In [43]:
include("../rollout.jl")
# Solve the acquisition function
# xnext, fnext = ei_solver(sur, lbs, ubs; initial_guesses=xstarts)
xnext, fnext = rollout_solver(sur=sur, tp=tp, xstarts=xstarts, batch=batch)
ynext = testfn(xnext)
# Update the surrogate model
sur = update_surrogate(sur, xnext, ynext)
sur = optimize_hypers_optim(sur, kernel_matern52)

P: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
History: [-391.15754060162743 0.0 0.0 0.0 0.0 0.0 0.0 499.99 0.0; 218.9031628453905 0.0 0.0 0.0 0.0 0.0 0.0 312.5 0.0; 82.66621969601522 0.0 0.0 0.0 0.0 0.0 0.0 -499.99 0.0; 431.0582938270935 0.0 0.0 0.0 0.0 0.0 0.0 -62.5 0.0] -- Xnext: [0.0, 0.0, 0.0, 0.0]
δxi_intermediates: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0] -- HEI: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0] -- EI: 0.0


LoadError: SingularException(1)

In [24]:
sur.X

4×7 Matrix{Float64}:
 -391.158   0.0  0.0  0.0  0.0  0.0  0.0
  218.903   0.0  0.0  0.0  0.0  0.0  0.0
   82.6662  0.0  0.0  0.0  0.0  0.0  0.0
  431.058   0.0  0.0  0.0  0.0  0.0  0.0